# Libraries and usefull functions

In [2]:
#Importing libraries

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import json


In [ ]:
#Urls to scrap

with open('urls_extractor/urls_extractor/spiders/urls.json', 'r') as file:
    datas = json.load(file)
    
urls_to_scrap = []

for data in datas :
    for key, value in data.items():
        urls_to_scrap.append(value)
        
urls_to_scrap

In [9]:
#Opening web browser
options = Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

#Getting the the page we want to scrap
driver.get('https://www.grainger.com/category/abrasives/abrasive-brushes/cup-brushes/angle-grinder-cup-brushes?categoryIndex=1')
driver.implicitly_wait(1)

#Extracting page main title
title_page = driver.find_element(By.CLASS_NAME, 'KkejIK').text

#Identification of each sections of the page. Creating an empty dictionary to save the data.
sections = driver.find_elements(By.CLASS_NAME, 'MAcbb-')
section_dict = {}

#Looping throught each section to get the information we want which are the tables
for section in sections :
    #Extracting section title
    title_section = section.find_element(By.CLASS_NAME, "sC0Aof").text
    
    sub_section_dict = {}
    section_dict[title_section] = sub_section_dict
    
    #Looping throught ever sub-section to extract every table
    sub_sections = section.find_elements(By.CLASS_NAME, "T8G8vu")
    for sub_section in sub_sections:
        title_sub_section = sub_section.find_element(By.CLASS_NAME, 'SQoGqa')
        sub_section_dict[title_sub_section.text] = {}
        
        #To load the HTML, we need to 'activate' the JS by clicking on a button of the table
        sub_section.find_element(By.CLASS_NAME, 'JxT10f').click()
        
        #Getting features name as they are different on each urls. Features comes in one long str thet needs to be split
        features = sub_section.find_element(By.CLASS_NAME, 'JxT10f')
        feature_list = (features.text).split('\n')
        
        #Getting the table associated with the sub section
        tables = sub_section.find_element(By.CLASS_NAME, 'cjpIYY')
        feature_dict = {}

        #Going through every row of every columns et creating the list of value assicaited to each feature
        rows = tables.find_elements(By.TAG_NAME, 'tr')
        for i in range(len(feature_list)):
            feature_dict[feature_list[i]] = []
            
            for row in rows:
                value = row.find_element(By.XPATH, f'./td[{i+1}]').text
                feature_dict[feature_list[i]].append(value)
                
        sub_section_dict[title_sub_section.text] = feature_dict

page_dict = {}
page_dict[title_page] = section_dict 

print(page_dict)

driver.quit()

{'Angle Grinder Cup Brushes': {'Light-Cleaning Brushes for Carbon Steel': {'3/8"-24 Arbor Hole': {'Brush Diameter': ['2-3/4 in', '3 in'], 'Bristle Diameter': ['0.02 in', '0.014 in'], 'Bristle Length': ['1 in', '1 in'], 'Brand': ['APPROVED VENDOR', 'WEILER'], 'Price': ['$7.54', '$30.21']}, '1/2"-13 Arbor Hole': {'Brush Diameter': ['3 in'], 'Bristle Diameter': ['0.014 in'], 'Bristle Length': ['1 in'], 'Brand': ['WEILER'], 'Price': ['$27.90']}, '5/8"-11 Arbor Hole': {'Brush Diameter': ['3 in', '3-1/2 in', '3-1/2 in', '2-3/4 in', '3 in', '3 in', '4 in', '5 in', '5 in', '6 in', '6 in', '4 in', '4 in', '5 in'], 'Bristle Diameter': ['0.014 in', '0.014 in', '0.014 in', '0.014 in', '0.014 in', '0.014 in', '0.014 in', '0.014 in', '0.02 in', '0.014 in', '0.02 in', '0.014 in', '0.02 in', '0.014 in'], 'Bristle Length': ['13/16 in', '7/8 in', '7/8 in', '1 in', '1 in', '1 in', '1-1/8 in', '1-1/4 in', '1-1/4 in', '1-1/4 in', '1-1/4 in', '1-3/8 in', '1-3/8 in', '1-3/8 in'], 'Brand': ['APPROVED VENDOR',

In [ ]:
#Saving to json

json_string = json.dumps(page_dict)

# Écrire la chaîne JSON dans un fichier
with open('mon_fichier.json', 'w') as datas:
    datas.write(json_string)